In [1]:
import time
import math 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re

In [3]:
browser = Browser('edge')

In [4]:
url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=20"
browser.visit(url)

In [5]:
html = browser.html
html_soup = soup(html, 'html.parser')

In [7]:
product_info = html_soup.find_all(class_='nl-product__content')
product_data1 = []

for product in product_info:
    product_name_element = product.find('div', class_='nl-product-card__title')
    original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
    clearance_price_element = product.find('span', class_='nl-price--total--red')
    rating_element = product.find('div', class_='bv_text')
    image_element = product.find('div', class_='nl-product-card__image-wrap')
    anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)

    product_name = product_name_element.text.strip() if product_name_element else 'N/A'
    original_price = original_price_element.text.strip() if original_price_element else 'N/A'
    clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
    rating = rating_element.text.strip() if rating_element else None
    product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
    product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
    product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'

    if original_price == 'N/A':
        alt_original_price_element = product.find('span', class_='nl-price--total')
        original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

    clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
    original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

    product_data1.append({
        'Product Name': product_name,
        'Original Price': original_price,
        'Clearance Price': clearance_price,
        'Rating': rating,
        'Product Code': product_code,
        'Link': product_link
    })

print(product_data1)

[{'Product Name': 'CCM FitLite 3DS Hockey Helmet Sticker/Decal Kit', 'Original Price': 9.99, 'Clearance Price': 1.93, 'Rating': None, 'Product Code': '083-8800-0', 'Link': 'https://www.canadiantire.ca/en/pdp/ccm-fitlite-3ds-hockey-helmet-sticker-decal-kit-0838800p.html#srp'}, {'Product Name': 'KidKusion Gummi Crib Rail Cover', 'Original Price': 21.99, 'Clearance Price': 7.93, 'Rating': None, 'Product Code': '046-8997-2', 'Link': 'https://www.canadiantire.ca/en/pdp/kidkusion-gummi-crib-rail-cover-0468997p.html#srp'}, {'Product Name': 'CANVAS Rubberized Ceramic Bathroom/Vanity Tray', 'Original Price': 13.99, 'Clearance Price': 7.93, 'Rating': None, 'Product Code': '063-6232-2', 'Link': 'https://www.canadiantire.ca/en/pdp/canvas-rubberized-ceramic-bathroom-vanity-tray-0636232p.html#srp'}, {'Product Name': 'For Living Acrylic Toothbrush Holder, Clear', 'Original Price': 8.99, 'Clearance Price': 4.93, 'Rating': None, 'Product Code': '063-2245-6', 'Link': 'https://www.canadiantire.ca/en/pdp/

In [8]:
df1 = pd.DataFrame(product_data1)

df1.sample(24)

,Product Name,Original Price,Clearance Price,Rating,Product Code,Link
18,"CANVAS Cloud Print Light Filtering Curtain, 2-...",44.99,22.93,None,046-6857-8,https://www.canadiantire.ca/en/pdp/canvas-clou...
6,ClosetMaid Suite Symphony Laminate Wood Top Sh...,29.99,14.93,None,068-7358-8,https://www.canadiantire.ca/en/pdp/closetmaid-...
2,CANVAS Rubberized Ceramic Bathroom/Vanity Tray,13.99,7.93,None,063-6232-2,https://www.canadiantire.ca/en/pdp/canvas-rubb...
15,Berkley Fusion 19 Octopus Hooks,5.29,2.43,None,077-7717X,https://www.canadiantire.ca/en/pdp/berkley-fus...
0,CCM FitLite 3DS Hockey Helmet Sticker/Decal Kit,9.99,1.93,None,083-8800-0,https://www.canadiantire.ca/en/pdp/ccm-fitlite...
13,Bounty Hunter Quick Draw™ Pro Metal Detector,379.99,224.93,None,079-4101-0,https://www.canadiantire.ca/en/pdp/bounty-hunt...
3,"For Living Acrylic Toothbrush Holder, Clear",8.99,4.93,None,063-2245-6,https://www.canadiantire.ca/en/pdp/for-living-...
7,"Perky Pet Squirrel Stumper Wild Bird Feeder, 3-lb",24.99,14.93,None,142-0762-8,https://www.canadiantire.ca/en/pdp/perky-pet-s...
17,"SIMONIZ Waterless Car Wash & Wax Spray, 454-g",13.99,6.93,None,039-2520-0,https://www.canadiantire.ca/en/pdp/simoniz-wat...
14,Brother TZE335 White on Black Laminated Label ...,29.99,17.93,None,069-4789-8,https://www.canadiantire.ca/en/pdp/brother-tze...


In [ ]:
df1['Percentage Off'] = round((df1['Clearance Price']/df1['Original Price'])*100)

df1.sample(24)

In [ ]:
base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service(
    "C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(base_url)
time.sleep(5)

html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(
        class_='nl-product-card__primary-details')

    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code
        })

driver.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
clearance_df = pd.DataFrame(product_data)
clearance_df.info()

In [ ]:
clearance_df['Percentage Off'] = round((clearance_df['Clearance Price']/clearance_df['Original Price'])*100)

clearance_df.sample(24)

In [ ]:
base_url = "https://www.canadiantire.ca/en/promotions/hot-sale.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver2 = webdriver.Chrome(service=service)

driver2.get(base_url)
time.sleep(5)
html_content = driver2.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver2.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver2.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info = html_soup.find_all(class_='nl-product-card__primary-details')

    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code
        })

driver2.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
sale_df = pd.DataFrame(sale_product_data)
sale_df['Percentage Off'] = round((sale_df['Sale Price']/sale_df['Original Price'])*100)
sale_df.info()

In [ ]:
base_url = "https://www.canadiantire.ca/en/promotions/red-alert-deals.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver3 = webdriver.Chrome(service=service)

driver3.get(base_url)
time.sleep(5)
html_content = driver3.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data_2 = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver3.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver3, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver3.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info2 = html_soup.find_all(class_='nl-product-card__primary-details')

    for product in sale_product_info2:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data_2.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code
        })

driver3.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
sale_df2 = pd.DataFrame(sale_product_data_2)
sale_df2['Percentage Off'] = round((sale_df2['Sale Price']/sale_df2['Original Price'])*100)

In [ ]:
sale_df2.info()

In [ ]:
sale_df2.sample(24)

In [ ]:
clearance_df.to_csv("./clearance.csv", index=True)
sale_df.to_csv("./sale.csv", index=True)
sale_df2.to_csv("./sale2.csv", index=True)